In [1]:
from gnutools import fs
from bpd import cfg
from bpd.dask import DataFrame, udf
from bpd.dask import functions as F
from gnutools.remote import gdrivezip
from bpd.dask.pipelines import *

In [2]:
# Import a sample dataset
df = DataFrame({"filename": fs.listfiles(gdrivezip(cfg.gdrive.google_mini)[0], [".wav"])})
df.compute()      

,filename
0,/tmp/1y4gwaS7LjYUhwTex1-lNHJJ71nLEh3fE/wow/919...
1,/tmp/1y4gwaS7LjYUhwTex1-lNHJJ71nLEh3fE/wow/6a2...
2,/tmp/1y4gwaS7LjYUhwTex1-lNHJJ71nLEh3fE/wow/682...
3,/tmp/1y4gwaS7LjYUhwTex1-lNHJJ71nLEh3fE/wow/beb...
4,/tmp/1y4gwaS7LjYUhwTex1-lNHJJ71nLEh3fE/wow/d37...


In [3]:
# Register a user-defined function
@udf
def word(f):
    return fs.name(fs.parent(f))

@udf
def initial(classe):
    return classe[0]

@udf
def lists(classes):
    return list(set(classes))
    

df.run_pipelines(
    [
        {
            select_cols: ("filename",),
            pipeline: (
                ("classe", word(F.col("filename"))),
                ("name", udf(fs.name)(F.col("filename"))),
            ),
        },
        {
            group_on: "classe",
            select_cols: ("name", ),
            pipeline: (
                ("initial", initial(F.col("classe"))),
            ),
        },
        {
            group_on: "initial",
            select_cols: ("classe", ),
            pipeline: (
                ("_initial", lists(F.col("classe"))),
            ),
        },
    ]
)\
.withColumnRenamed("_initial", "initial")\
.compute()

,filename,initial
0,/tmp/1y4gwaS7LjYUhwTex1-lNHJJ71nLEh3fE/wow/919...,[wow]
0,/tmp/1y4gwaS7LjYUhwTex1-lNHJJ71nLEh3fE/wow/6a2...,[wow]
0,/tmp/1y4gwaS7LjYUhwTex1-lNHJJ71nLEh3fE/wow/682...,[wow]
0,/tmp/1y4gwaS7LjYUhwTex1-lNHJJ71nLEh3fE/wow/beb...,[wow]
0,/tmp/1y4gwaS7LjYUhwTex1-lNHJJ71nLEh3fE/wow/d37...,[wow]
